In [1]:
import numpy as np
import gym
import matplotlib.pyplot as plt
from dm_control.suite.wrappers import pixels
from dm_control import suite

import dataset as ds#my file
import vae as v#my file
import parameters #my file
import lstm_validation as ls #my file
import learned_env as le

import matplotlib.pyplot as plt
%matplotlib inline
%matplotlib notebook

import os
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"   # see issue #152
os.environ["CUDA_VISIBLE_DEVICES"]="0"

/home/luca/anaconda3/envs/drl/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/luca/anaconda3/envs/drl/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/luca/anaconda3/envs/drl/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/luca/anaconda3/envs/drl/lib/python3.7/site-packages/ten

In [2]:
#Tensorflow version: 1.14.0

#ENVIRONMENT
env = suite.load("reacher","easy",visualize_reward=True)
action_space = env.action_spec().shape[0]

# DATASET
dataset = ds.Dataset(env) 
dataset.load_dataset()

#VAE
vae = v.VAE(dataset)
vae.load_json()


# LSTM 
lstm_train = ls.LSTM(action_space)
batch_encoded_frames,batch_actions,batch_rewards,batch_reset = dataset.split_set_for_lstm("train_set")


Instructions for updating:
Use `tf.keras.layers.Conv2D` instead.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


Dataset of observations is loaded:  (200, 500, 64, 64, 3)
Loaded current datasets:  dict_keys(['act', 'rew', 'reset', 'obs', 'enc_obs'])


Instructions for updating:
Use keras.layers.flatten instead.
Instructions for updating:
Use keras.layers.dense instead.


Instructions for updating:
Use `tf.keras.layers.Conv2DTranspose` instead.


Instructions for updating:
This class is equivalent as tf.keras.layers.LSTMCell, and will be replaced by that in Tensorflow 2.0.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


Model loaded!


Session Started!
<tf.Variable 'lstm_unroll/lstm_cell/kernel:0' shape=(580, 2048) dtype=float32_ref>
<tf.Variable 'lstm_unroll/lstm_cell/bias:0' shape=(2048,) dtype=float32_ref>
<tf.Variable 'lstm_activation_function/rnn_output_w:0' shape=(512, 66) dtype=float32_ref>
<tf.Variable 'lstm_activation_function/rnn_output_b:0' shape=(66,) dtype=float32_ref>


In [ ]:
# TRAINING DI LSTM

#lstm.load()
lstm_train.train_lstm_mdn(batch_encoded_frames,batch_actions,batch_rewards,batch_reset, ELIMINAMI = 0.004, EPOCHE = 10)

In [4]:
lstm_inf = ls.LSTM(action_space, seq_len = 2, batch_size = 1)
lstm_inf.load_json()

Session Started!
<tf.Variable 'lstm_unroll/lstm_cell/kernel:0' shape=(580, 2048) dtype=float32_ref>
<tf.Variable 'lstm_unroll/lstm_cell/bias:0' shape=(2048,) dtype=float32_ref>
<tf.Variable 'lstm_activation_function/rnn_output_w:0' shape=(512, 66) dtype=float32_ref>
<tf.Variable 'lstm_activation_function/rnn_output_b:0' shape=(66,) dtype=float32_ref>
Model loaded!


In [21]:
#Initialize fist lstm state
lstm_state = lstm_inf.sess.run(lstm_inf.initial_state)

# Peek a random frame into dataset and encode it
idx = 5
frame = dataset.dataset["obs"][1][idx]
encoded_frame = vae.encode_input(frame)

# Reshape placeholders 
prev_z = np.zeros((1, 1, parameters.LATENT_SIZE))
prev_z[0][0] = encoded_frame

prev_action = np.zeros((1, 1,action_space))
prev_action[0] = dataset.dataset["act"][1][idx]


prev_rew = np.zeros((1,1))
prev_rew[0] = dataset.dataset["rew"][1][idx]

prev_restart = np.ones((1, 1))
prev_restart[0] = dataset.dataset["reset"][1][idx]


# Feed placeholders
feed = {
    lstm_inf.input_obs: prev_z,
    lstm_inf.input_action: prev_action,
    lstm_inf.input_rewards: prev_rew,
    lstm_inf.input_res_flag: prev_restart,
    lstm_inf.initial_state: lstm_state,
}

# Do 1-step inference
new_enc_frame,done,rew,lstm_state = lstm_inf.sess.run([lstm_inf.target_obs,lstm_inf.target_restart,lstm_inf.target_rewards,lstm_inf.next_state], feed)

# Decode producted frames
decoded_frame = vae.decode_latent_vec(new_enc_frame)
reconstructed_frame = np.round(reconstructed_frame * 255.).astype(np.uint8)
reconstructed_frame = reconstructed_frame.reshape(64, 64, 3) #remove batch dimension

# Show results
plt.figure(3)
plt.imshow(reconstructed_frame)
plt.show()

print("Done: ",done, " should be: ", dataset.dataset["res"][1][idx])
print("Reward: ",rew," should be: ", dataset.dataset["rew"][1][idx])


InvalidArgumentError: You must feed a value for placeholder tensor 'batch_enc_frames' with dtype float and shape [1,2,64]
	 [[node batch_enc_frames (defined at /home/luca/Desktop/drlux_plan_pix/lstm_validation.py:288) ]]

Original stack trace for 'batch_enc_frames':
  File "/home/luca/anaconda3/envs/drl/lib/python3.7/runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "/home/luca/anaconda3/envs/drl/lib/python3.7/runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "/home/luca/anaconda3/envs/drl/lib/python3.7/site-packages/ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()
  File "/home/luca/anaconda3/envs/drl/lib/python3.7/site-packages/traitlets/config/application.py", line 664, in launch_instance
    app.start()
  File "/home/luca/anaconda3/envs/drl/lib/python3.7/site-packages/ipykernel/kernelapp.py", line 583, in start
    self.io_loop.start()
  File "/home/luca/anaconda3/envs/drl/lib/python3.7/site-packages/tornado/pla

In [7]:
print(action_space)

2
